# PROJECT 3: SMART ILLUMINATION - PRELIMINARY
In this project, you will investigate the use of an LED array to generate multiple imaging modalities using the same microscope. Contrast generation is one of the key aspects of microscopy that enables useful images to be acquired. Contrast can be generated by employing a number of different illumination techniques. A custom-addressable diode array allows flexible control of sample illumination and can therefore be used to enhance contrast in a number of ways.

## Initial variable and paths definitions
We are going to first define some global variables and paths.

In [ ]:
import os
img_folder = "images_preliminary"
if not os.path.exists(img_folder):
    os.makedirs(img_folder)

## INTIAL IMPORTS

In [ ]:
from picamera2 import Picamera2, Preview
from RPiLedMatrix import RPiLedMatrix
import time
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import cv2

## IMPORT, START, AND TEST LED ARRAY

This code imports the camera and runs a script for visual inspection that all LEDs are lighting up. For more information see the source code in RPiLedMatrix.py

In [ ]:
# Initialize the led matrix
led_matrix = RPiLedMatrix()
led_matrix.set_rotation(0)

led_matrix.check_leds_visual()

# Close the led matrix
led_matrix.off()
led_matrix.close()

This one runs over a series of shapes to check that it all works properly. For more information see the source code in RPiLedMatrix.py

In [ ]:
# Initialize the led matrix
led_matrix = RPiLedMatrix()
led_matrix.set_rotation(0)

led_matrix.check_shapes()

# Close the led matrix
led_matrix.off()
led_matrix.close()

<font color='orange'>NOW GO BACK TO THE INSTRUCTIONS AND ASSEMBLE THE OPTICAL ELEMENTS</font> 

## ALIGN OPTICAL ELEMENTS

At this point, the microscope should be completely assembled.

Use the following script to light up the array and take a live preview of the camera. Using this, set up the relative distances of the elements to achieve imaging. Some information about the working distance and focal lengths of the elements can be found in:

* Objecive WD = 18.5 mm https://www.edmundoptics.com/p/olympus-pln-4x-objective/29221/
* Tube Lens (AC254-075-AB-ML) fb = 68.72 mm (see reference drawing in https://www.thorlabs.com/newgrouppage9.cfm?objectgroup_id=12804)

<font color='orange'>Use this information to set up the distance between the optical elements to form an image of whichever sample slide you decide to take (we recommend a slide with a large sample to begin with). Ensure that it is mounted such that you have an **infinity corrected imaging system** (i.e., there is an infinite plane between the objective and the tube lens). It might be good to look at the following reference: </font>

In this case, the auto-exposure is enabled, so no need to tune that parameter. You can tune the brightness of the array if you wish to. Remember to COVER the setup to prevent ambient light from hitting the camera chip. 

At this point, do not worry about the code itself, we will dive into some of the details later in this scritp.

In [ ]:
# Initialize the Picamera2 object
rpicamera = Picamera2()

# Start the preview using the QTGL backend
rpicamera.start_preview(Preview.QTGL)

# Define camera controls with auto exposure enabled
controls = {"AeEnable": True}

# Create a preview configuration with the specified controls
preview_config = rpicamera.create_preview_configuration(controls=controls)

# Apply the preview configuration to the camera
rpicamera.configure(preview_config)

# Start the camera
rpicamera.start()

# Allow the camera to warm up
time.sleep(0.1)

# Initialize the LED matrix
led_matrix = RPiLedMatrix()

# Set the LED matrix rotation to 0 degrees
led_matrix.set_rotation(0)

# Turn on all LEDs with 100% brightness
led_matrix.on_all(brightness=1)

# Wait for user input to close the application
ans = input("Press enter to close")

# Turn off all LEDs
led_matrix.off()

# Close the LED matrix object
led_matrix.close()

# Stop the camera
rpicamera.stop()

# Close the camera object
rpicamera.close()

## TAKE SINGLE IMAGE

Now that the system is working, we will work on the collection of images. The picamera2 library that we are going to be using has a large number of modalities and options to enable different collection modes. The complete guide of the library can be found in https://datasheets.raspberrypi.com/camera/picamera2-manual.pdf (the PDF is also in this folder).

In general, the workflow for image/video collection is as follows:
* Initialize the camera (rpicamera = Picamera2())
* Configure the camera settings -- we will discuss about this later
* Start the camera (rpicamera.start())
* Do operation
* Stop the camera (rpicamera.stop())
* Close the camera (rpicamera.close())



### CAMERA CONFIGURATION
To configure the camera, picamera2 allows to predefine three sets of configurations, the preview configuration (for low-resolution images and preview videos) and the still configuration (for high resolution images), and the video configuration (for video recording). In this module, we will focus on the first two only, as we will not be taking videos. 

To configure these modes, we use the following scripts:

_preview_config = rpicamera.create_preview_configuration()_

_still_config = rpicamera.create_still_configuration()_

_rpicamera.configure(preview_config) OR rpicamera.configure(still_config)_

The previous lines would load the default or last configuration that was set up. In the configurations, a large number of parameters can be changed. A detail documentation is provided in the library documentation (https://datasheets.raspberrypi.com/camera/picamera2-manual.pdf -- the PDF is also in this folder) and many very informative examples are provided in https://github.com/raspberrypi/picamera2.

In the context of this module, we will only see how to change the following parameters (in these examples it is using the preview configuration, but it can be done in either):

#### ENABLE/DISABLE AUTO-EXPOSURE
_preview_config = rpicamera.create_preview_configuration(main={controls = {"AeEnable": True})_

_rpicamera.configure(preview_config)_

#### SET IMAGE SIZE
_preview_config = rpicamera.create_preview_configuration(main={"size": (600, 600)})_

_rpicamera.configure(preview_config)_

### SET EXPOSURE AND GAIN
_preview_config = rpicamera.create_preview_configuration(controls = {"ExposureTime": 50000, "AnalogueGain": 1.0})_

_rpicamera.configure(preview_config)_


Of course, these parameters can all be changed in parellel. In the following, we have provided some examples:

## CAPTURE SINGLE LOW-Q IMAGE

To take low resolution images we will use the preview configuration. In this particular example, we will be using the Auto Exposure mode. 

In [ ]:
# Define the image name to save the captured image
image_name = "test_LQ.png"

# Initialize the Picamera2 object
rpicamera = Picamera2()

# Create a preview configuration with a specified size and controls
preview_config = rpicamera.create_preview_configuration(main={"size": (600, 600)}, controls={"AeEnable": True})

# Apply the preview configuration to the camera
rpicamera.configure(preview_config)

# Start the camera
rpicamera.start()

# Initialize the LED matrix
led_matrix = RPiLedMatrix()

# Set the LED matrix rotation to 0 degrees
led_matrix.set_rotation(0)

# Turn on all LEDs with 50% brightness
led_matrix.on_all(brightness=0.5)

# Allow the camera and LEDs to stabilize
time.sleep(2)

# Capture an image and save it to the specified file, also retrieve metadata
metadata = rpicamera.capture_file(os.path.join(img_folder,image_name))
print(metadata)

# Stop the camera
rpicamera.stop()

# Close the camera object
rpicamera.close()

# Turn off all LEDs
led_matrix.off()

# Close the LED matrix object
led_matrix.close()

# Plot the captured image using matplotlib
plt.figure(figsize=(8, 8))
plt.imshow(np.array(Image.open(os.path.join(img_folder,image_name))))
plt.axis('off')  # Hide the axis
plt.title(f"Loaded Image: {image_name}")
# plt.show()

## CAPTURE SINGLE HIGH-Q IMAGE

To take high resolution images we will use the still configuration. In this particular example, we will be using the Auto Exposure mode.

In [ ]:
image_name = "test_HQ.png"

rpicamera = Picamera2()
capture_config = rpicamera.create_still_configuration(main={"size": (2000, 2000)}, controls = {"AeEnable": True})

rpicamera.configure(capture_config)

rpicamera.start()
led_matrix = RPiLedMatrix()
led_matrix.set_rotation(0)

led_matrix.on_all(brightness=0.5)

time.sleep(2)

metadata = rpicamera.capture_file(os.path.join(img_folder,image_name))
print(metadata)

rpicamera.stop()
rpicamera.close()

led_matrix.off()
led_matrix.close()


# Plot the image using matplotlib
plt.figure(figsize=(8,8))
plt.imshow(np.array(Image.open(os.path.join(img_folder,image_name))))
plt.axis('off')  # Hide the axis
plt.title(f"Loaded Image: {image_name}")
# plt.show()

## CHANGE EXPOSURE TIMES

In this example we take to images with different exposure times but same gain. Although we will not be using Gain changes in this example, EdmundOptics provides a good intro to what is gain in digital cameras (https://www.edmundoptics.com/knowledge-center/application-notes/imaging/basics-of-digital-camera-settings-for-improved-imaging-results/) 

In [ ]:
# Initialize the Picamera2 object
rpicamera = Picamera2()

# Create still configuration for low exposure
preview_config_lowEXP = rpicamera.create_still_configuration(main={"size": (2000, 2000)}, controls={"ExposureTime": 10000, "AnalogueGain": 1.0})

# Create still configuration for high exposure
preview_config_highEXP = rpicamera.create_still_configuration(main={"size": (2000, 2000)}, controls={"ExposureTime": 80000, "AnalogueGain": 1.0})

# Set the LED brightness level
brightness = 0.7

# Configure the camera for low exposure settings
rpicamera.configure(preview_config_lowEXP)

# Start the camera
rpicamera.start()

# Initialize the LED matrix
led_matrix = RPiLedMatrix()

# Set the LED matrix rotation to 0 degrees
led_matrix.set_rotation(0)

# Turn on all LEDs with the specified brightness
led_matrix.on_all(brightness=brightness)

# Allow the camera and LEDs to stabilize
time.sleep(1)

# Capture an image with low exposure settings and save it, also retrieve metadata
metadata = rpicamera.capture_file(os.path.join(img_folder,"test_HQ_LEXP.png"))
print(metadata)

# Stop the camera
rpicamera.stop()

# Configure the camera for high exposure settings
rpicamera.configure(preview_config_highEXP)

# Start the camera again
rpicamera.start()

# Re-initialize the LED matrix
led_matrix = RPiLedMatrix()

# Set the LED matrix rotation to 0 degrees
led_matrix.set_rotation(0)

# Turn on all LEDs with the specified brightness
led_matrix.on_all(brightness=brightness)

# Allow the camera and LEDs to stabilize
time.sleep(1)

# Capture an image with high exposure settings and save it, also retrieve metadata
metadata = rpicamera.capture_file(os.path.join(img_folder,"test_HQ_HEXP.png"))
print(metadata)

# Stop the camera
rpicamera.stop()

# Close the camera object
rpicamera.close()

# Turn off all LEDs
led_matrix.off()

# Close the LED matrix object
led_matrix.close()

# Plot the images using matplotlib
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Plot the low exposure image
ax[0].imshow(np.array(Image.open(os.path.join(img_folder,"test_HQ_LEXP.png"))))
ax[0].axis('off')  # Hide the axis
ax[0].set_title("test_HQ_LowEXP")

# Plot the high exposure image
ax[1].imshow(np.array(Image.open(os.path.join(img_folder,"test_HQ_HEXP.png"))))
ax[1].axis('off')  # Hide the axis
ax[1].set_title("test_HQ_HighEXP")



## GET IMAGE ARRAY

To work on postprocessing, you probably want to take the image as arrays (ndarray in Python) to then perform operations on these matrices. Also, we will see a feature that can be useful, the _switch_mode_and_capture_array()_ which will enable to take high resolution images (using the still_config) while being in the preview, without having to change modes.

In this particular case, in the preview once the user presses ENTER it takes the image in high resolution and saves it as an array.

In [ ]:
# Import and initialize the Picamera2 instance
rpicamera = Picamera2()

# Start the camera preview with QTGL backend
rpicamera.start_preview(Preview.QTGL)

# Create a preview configuration with specified size and automatic exposure enabled
preview_config = rpicamera.create_preview_configuration(main={"size": (600, 600)}, controls = {"AeEnable": True})

# Create a capture configuration with specified size and automatic exposure enabled
capture_config = rpicamera.create_still_configuration(main={"size": (2000, 2000)}, controls = {"AeEnable": True})

# Configure the camera with the preview configuration
rpicamera.configure(preview_config)

# Start the camera
rpicamera.start()

# Initialize the LED matrix and set its rotation
led_matrix = RPiLedMatrix()
led_matrix.set_rotation(0)

# Turn on all LEDs with 40% brightness
led_matrix.on_all(brightness=0.4)

# Wait for 1 second
time.sleep(1)

# Prompt the user to press enter to capture an image
ans = input("Press enter to capture array")

# Switch to capture mode and capture the image as an array
img_array = rpicamera.switch_mode_and_capture_array(capture_config, "main")

# Print the shape of the captured image array
print(img_array.shape)

# Stop and close the camera
rpicamera.stop()
rpicamera.close()

# Turn off and close the LED matrix
led_matrix.off()
led_matrix.close()

### PLOT THE IMAGE
If you want to plot that image you can do so:

In [ ]:
import matplotlib.pyplot as plt

# Plot the image using matplotlib
plt.figure(figsize=(8,8))
plt.imshow(img_array)
plt.axis('off')  # Hide the axis
plt.title("Color Image")

**IMPORTANT** note that in this case, the channels of the array might not correspond to what we want:

In [ ]:
print("Average intensity channel 1:", img_array[:,:,0].mean())
print("Average intensity channel 2:", img_array[:,:,1].mean())
print("Average intensity channel 3:", img_array[:,:,2].mean())

### SAVE ARRAY AS IMAGE
You can also save that image:

In [ ]:
# Convert the ndarray to a PIL Image
image = Image.fromarray(img_array)

# Save the image as a PNG file
image.save(os.path.join(img_folder,'test_load.png'))

### LOAD IMAGE AS ARRAY
Finally, you can also load a saved image into an array:

In [ ]:
# Load an image from a path
image_name = 'test_load.png'  # Replace with the correct path to your image file
image = Image.open(os.path.join(img_folder,image_name))

# Convert the image to a numpy array
image_array = np.array(image)

# Plot the image using matplotlib
plt.figure(figsize=(8,8))
plt.imshow(image_array)
plt.axis('off')  # Hide the axis
plt.title(f"Loaded Image: {image_name}")
plt.show()

In this case, when using Pillow to load the image, the channels are:

In [ ]:
print("Average intensity channel 1:", image_array[:,:,0].mean())
print("Average intensity channel 2:", image_array[:,:,1].mean())
print("Average intensity channel 3:", image_array[:,:,2].mean())

## CONVERT TO GRAYSCALE
It might be interesting to convert the images to grayscale at some point. To do so you can use pillow and open-cv libraries.

In [ ]:
def convert_to_grayscale(image_array):
    image = Image.fromarray(image_array.astype(np.uint8))
    grayscale_image = image.convert("L")
    return np.array(grayscale_image)


# Plot the image using matplotlib
fig, ax = plt.subplots(1,2, figsize=(12,6))
ax[0].imshow(convert_to_grayscale(image_array), cmap='gray')
ax[0].axis('off')  # Hide the axis
ax[0].set_title("Grayscale Image - PILLOW")


# Plot the image using matplotlib
ax[1].imshow(cv2.cvtColor(cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR), cv2.COLOR_BGR2GRAY), cmap='gray')
ax[1].axis('off')  # Hide the axis
ax[1].set_title("Grayscale Image - CV2")

# YOU'RE UP!
Now explore different types of illuminations and processing of images. For that, you might find usefu the examples in: https://github.com/raspberrypi/picamera2

# EXTRA: USING THE PICAMERA2 APP
For experimenting with exposure times and setups, it might be useful to use the app developped by Raspberry Pi (https://github.com/raspberrypi/picamera2/apps) and that we have adapted to make it callable from different scripts (see RPiCameraApp.py). A usage example is provided below.

In [ ]:
from RPiCameraApp import RPiCameraApp
from RPiLedMatrix import RPiLedMatrix

# Initialize the LED matrix
led_matrix = RPiLedMatrix()

# Set the LED matrix rotation to 0 degrees
led_matrix.set_rotation(0)

# Turn on all LEDs with 20% brightness
led_matrix.on_all(brightness=1)

# Run the app
camera_app = RPiCameraApp()
camera_app.run()

# Turn off all LEDs
led_matrix.off()

# Close the LED matrix object
led_matrix.close()


# EXTRA: FOURIER TRANSFORM AND FILTERING IN FOURIER SPACE

### Introduction to Image Processing with Fourier Transform and Bandpass Filtering

This guide will introduce you to basic concepts in image processing, specifically focusing on the Fourier Transform and the application of a bandpass filter. Understanding these concepts will enhance your ability to analyze and manipulate images in the frequency domain, which can be particularly useful in various fields such as signal processing, medical imaging, and computer vision.

#### Key Concepts

1. **Image Representation:**
   - Digital images are represented as matrices of pixel values. Each pixel value corresponds to the intensity of light at that point in the image. For grayscale images, these values range from 0 (black) to 255 (white).

2. **Grayscale Conversion:**
   - Converting an image to grayscale simplifies processing by reducing the image to a single channel. This is often a preliminary step in image analysis.

3. **Fourier Transform:**
   - The Fourier Transform is a mathematical tool that transforms an image from the spatial domain to the frequency domain. It decomposes the image into its sinusoidal components, allowing analysis of the image's frequency characteristics.
   - The 2D Fourier Transform is used for images, providing a frequency representation where low frequencies are concentrated at the center and high frequencies are towards the edges.

4. **Magnitude Spectrum:**
   - The magnitude spectrum represents the amplitude of different frequency components. It is often visualized using a logarithmic scale to handle the wide range of values.

5. **Bandpass Filtering:**
   - A bandpass filter allows frequencies within a certain range to pass through while attenuating frequencies outside this range. This is useful for removing noise or isolating specific features in the image.

6. **Inverse Fourier Transform:**
   - To convert the filtered frequency representation back to the spatial domain, the Inverse Fourier Transform is applied. This reconstructs the image from its frequency components.

#### What the Code Does

1. **Load and Convert Image:**
   - The image is loaded and converted to grayscale using the `load_image` function. This simplifies the data to a 2D array of pixel values.

2. **Compute Fourier Transform:**
   - The `compute_fourier_transform` function computes the 2D Fourier Transform of the grayscale image, shifts the zero frequency component to the center, and calculates the magnitude spectrum for visualization.

3. **Apply Bandpass Filter:**
   - The `apply_bandpass_filter` function creates a mask to filter out frequencies outside the specified range (low and high cutoff frequencies) and applies this mask to the shifted Fourier Transform.

4. **Visualize Results:**
   - The `plot_images` function displays the original image, its Fourier Transform, the filtered image, and the Fourier Transform of the filtered image. This provides a clear comparison of the effects of the bandpass filter.

#### Using the Code

- Ensure you have the necessary libraries installed: `numpy`, `matplotlib`, and `PIL`.
- Set the path to your image and define the low and high cutoff frequencies.
- Run the code to load the image, compute and apply the Fourier Transform and bandpass filter, and visualize the results.

This process highlights how frequency domain analysis can be used to filter and enhance images, providing powerful tools for various image processing applications.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

# Function to load the image and convert it to grayscale
def load_image(image_path):
    image = Image.open(image_path).convert('L')
    return np.array(image)

# Function to compute the Fourier Transform of the image
def compute_fourier_transform(image):
    f_transform = np.fft.fft2(image)
    f_shifted = np.fft.fftshift(f_transform)  # Shift the zero frequency component to the center
    magnitude_spectrum = 20 * np.log(np.abs(f_shifted) + 1)  # Compute the magnitude spectrum
    return f_shifted, magnitude_spectrum

# Function to apply a bandpass filter in the Fourier domain
def apply_bandpass_filter(f_shifted, low_cutoff, high_cutoff):
    rows, cols = f_shifted.shape
    crow, ccol = rows // 2 , cols // 2  # Center of the image

    mask = np.zeros((rows, cols), dtype=np.uint8)
    mask[crow-high_cutoff:crow+high_cutoff, ccol-high_cutoff:ccol+high_cutoff] = 1
    mask[crow-low_cutoff:crow+low_cutoff, ccol-low_cutoff:ccol+low_cutoff] = 0

    f_filtered = f_shifted * mask
    return f_filtered

# Function to plot the original and filtered images along with their Fourier Transforms
def plot_images(original, original_ft, filtered, filtered_ft):
    fig, ax = plt.subplots(2, 2, figsize=(12, 12))
    
    # Plot original image
    ax[0, 0].imshow(original, cmap='gray')
    ax[0, 0].set_title('Original Image')
    ax[0, 0].axis('off')
    
    # Plot Fourier Transform of original image
    ax[0, 1].imshow(original_ft, cmap='gray')
    ax[0, 1].set_title('Fourier Transform of Original')
    # Center the axis
    rows, cols = original_ft.shape
    ax[0, 1].set_xticks(np.arange(0, cols, cols//4))
    ax[0, 1].set_yticks(np.arange(0, rows, rows//4))
    ax[0, 1].set_xticklabels(np.arange(-cols//2, cols//2, cols//4))
    ax[0, 1].set_yticklabels(np.arange(-rows//2, rows//2, rows//4))
    
    # Plot filtered image
    ax[1, 0].imshow(filtered, cmap='gray')
    ax[1, 0].set_title('Filtered Image')
    ax[1, 0].axis('off')
    
    # Plot Fourier Transform of filtered image
    ax[1, 1].imshow(filtered_ft, cmap='gray')
    ax[1, 1].set_title('Fourier Transform of Filtered')
    # Center the axis
    ax[1, 1].set_xticks(np.arange(0, cols, cols//4))
    ax[1, 1].set_yticks(np.arange(0, rows, rows//4))
    ax[1, 1].set_xticklabels(np.arange(-cols//2, cols//2, cols//4))
    ax[1, 1].set_yticklabels(np.arange(-rows//2, rows//2, rows//4))
    
    plt.show()

image_name = 'tilia_stem.png'
image_path = os.path.join('images_examples',image_name)
low_cutoff = 70  # Example low cutoff frequency
high_cutoff = 500  # Example high cutoff frequency

original_image = load_image(image_path)
f_shifted, original_ft = compute_fourier_transform(original_image)

f_filtered = apply_bandpass_filter(f_shifted, low_cutoff, high_cutoff)
filtered_image = np.fft.ifft2(np.fft.ifftshift(f_filtered)).real
filtered_ft = 20 * np.log(np.abs(f_filtered) + 1)  # Compute magnitude spectrum of filtered FT

plot_images(original_image, original_ft, filtered_image, filtered_ft)
